# Real experiment evaluation

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
def convert_room_to_robot(x_room):
    '''
    Convert coordinates in room reference to robot reference. 
    '''
    assert x_room.shape[0] == 3
    assert x_room.ndim > 1 and x_room.shape[1] >= 1
    
    from math import cos, sin, pi
    
    theta = pi/2.0
    R = np.array([[cos(theta), -sin(theta), 0], [sin(theta), cos(theta), 0], [0, 0, 1]])
    #origin = np.array([3.4, 3.58, 0.0]).reshape((3, 1))
    origin = np.array([3.58, 3.4, 0.0]).reshape((3, 1))
    x_robot = R.dot(x_room - origin)
    return x_robot
    
datafile = 'experiments/robot_test/circle2_double.csv'
anchorsfile = 'experiments/anchors.csv'

tango_system_id = 7585
rtt_system_id = 7592

In [ ]:
rtt_counter = -1
tango_counter = -1

def add_name(row):
    global rtt_counter
    global tango_counter 
    if row.system_id == tango_system_id:
        tango_counter += 1
        return 'Tango {}'.format(tango_counter)
    elif row.system_id == rtt_system_id:
        rtt_counter += 1
        return 'RTT {}'.format(rtt_counter)
    
def clean_anchor_id(row):
    return row.anchor_id.strip()

anchors_df = pd.read_csv(anchorsfile)
anchors_df.loc[:, 'anchor_id'] = anchors_df.apply(lambda row: clean_anchor_id(row),  axis=1)
anchors_df.loc[:, 'anchor_name'] = anchors_df.apply(lambda row: add_name(row), axis=1) 
anchors_df

# Distance filtering

In [ ]:
def change_name(row, anchors_df):
    return anchors_df.loc[anchors_df.anchor_id==row.anchor_id, "anchor_name"].values[0]

def change_system_id(row):
    if row.system_id==rtt_system_id:
        return 'RTT'
    elif row.system_id==tango_system_id:
        return 'Tango'
    
def filter_columns(data_df):
    all_columns = set(data_df.columns)
    keep_columns = set(['timestamp', 'system_id', 'anchor_id',
                    'px', 'py', 'pz',
                    'theta_x', 'theta_y', 'theta_z',
                    'distance', 'rssi', 'seconds'])
    drop_columns = all_columns - keep_columns # this is a set difference
    data_df.drop(drop_columns, axis=1, inplace=True)

def read_dataset(datafile):
    data_df = pd.read_csv(datafile)
    filter_columns(data_df)
    data_df.loc[:, 'timestamp'] = (data_df.timestamp.values - data_df.timestamp.min()) / 1000. # in seconds
    data_df.loc[:, 'anchor_name'] = data_df.apply(lambda row: change_name(row, anchors_df), axis=1)
    data_df.loc[:, 'system_id'] = data_df.apply(lambda row: change_system_id(row), axis=1)
    return data_df

## Resampling

In [ ]:
from evaluate_dataset import resample

datafile_name = datafile.split('.')[0]
resample_name = datafile_name + '_resampled.pkl'

create_new_results = False

try:
    assert not create_new_results, "Asked for result recreation."
    resampled_df = pd.read_pickle(resample_name)
    print('Read', resample_name)
except (FileNotFoundError, AssertionError) as e:
    print(e)
    print('Creating new results.'.format(resample_name))
    
    data_df = read_dataset(datafile)
    tmax = data_df.timestamp.max()
    tmin = data_df.timestamp.min()
    resampled_df_rtt = resample(data_df, t_range=[tmin, tmax], t_delta=0.5, t_window=1.0, system_id='RTT') 
    resampled_df_tango = resample(data_df, t_range=[tmin, tmax], t_delta=0.5, t_window=0.1, system_id='Tango') 
    resampled_df = pd.concat((resampled_df_rtt, resampled_df_tango), ignore_index=True)
    resampled_df['anchor_name'] = resampled_df.apply(lambda row: change_name(row, anchors_df), axis=1)
    resampled_df.sort_values(["timestamp", "anchor_name"], inplace=True)
    resampled_df.to_pickle(resample_name)
    print(resampled_df.head())
    print('Saved as', resample_name)

## Calibration

In [ ]:
# we can do below either with data_df or with resampled_df

def distance_gt(row, anchors_df, gt_system_id="Tango"):
    if row.system_id == gt_system_id:
        return 0.0
    anchor_coord = anchors_df.loc[anchors_df.anchor_id==row.anchor_id, ['px', 'py', 'pz']].values.astype(np.float32)
    point_coord = np.array([row.px, row.py, row.pz], dtype=np.float32)
    return np.linalg.norm(point_coord - anchor_coord)

from evaluate_dataset import add_gt_raw
data_df = read_dataset(datafile)
data_df = add_gt_raw(data_df, t_window=0.1)
data_df.loc[:, 'distance_tango'] = data_df.apply(lambda row: distance_gt(row, anchors_df), axis=1)

#from evaluate_dataset import add_gt_resampled
#data_df = resampled_df
#data_df = add_gt_resampled(data_df, anchors_df)
#data_df.head()

In [ ]:
def diff(rolling):
    if len(rolling) > 1:
        return rolling[1] - rolling[0]
    else: 
        return 0.0

def add_length(pos_df):
    import datetime
    pos_df.index = [datetime.datetime.fromtimestamp(t) for t in pos_df.timestamp]
    v_x = pos_df.loc[:, ["px"]].rolling(min_periods=1, window=2, center=False).apply(func=diff, raw=True)
    v_y = pos_df.loc[:, ["py"]].rolling(min_periods=1, window=2, center=False).apply(func=diff, raw=True)
    d_t = pos_df.loc[:, ["timestamp"]].rolling(min_periods=1, window=2, center=False).apply(func=diff, raw=True)
    pos_df.reset_index(inplace=True, drop=True)

    v = np.hstack((
        v_x.values.astype(np.float32),
        v_y.values.astype(np.float32)))
    lengths = np.linalg.norm(v, axis=1)
    
    # TODO this is numerically bad, velocities are extremely noisy...
    d_times = d_t.values.astype(np.float32).reshape((-1,))
    velocities = lengths / d_times
    median = np.median(velocities[d_times >= np.median(d_times)])
    velocities[d_times < np.median(d_times)] = median
    
    if plot:
        fig, axs= plt.subplots(3,1)
        fig.set_size_inches(15, 5)
        axs[0].plot(d_times, label='d_times')
        axs[1].plot(lengths, label='lengths')
        axs[2].plot(velocities, label='length')
        axs[2].set_ylim([0, 0.3])
    
    pos_df.loc[:, "length"] = lengths
    return pos_df

def find_edge(window, pattern=[1, 1, -1]):
    if len(window) < len(pattern):
        pattern = pattern[:-len(window)]
    sum_ = np.sum([a*b for (a, b) in zip(pattern, window)])
    return sum_
    
def find_start_times(tango_df, thresh_filter=-0.5, pattern=[1, 1, 1, 1, -1, -1], plot=False):
    if "length" not in tango_df.columns:
        tango_df = add_length(tango_df)
    
    tango_df.index = [datetime.datetime.fromtimestamp(t) for t in tango_df.timestamp]
    normalized_series = tango_df.loc[:, "length"]/tango_df.length.max()
    df = normalized_series.rolling(window=6, center=False, min_periods=1).apply(find_edge, raw=True, kwargs={'pattern':pattern})
    tango_df.reset_index(inplace=True, drop=True)
    
    if plot:
        plt.figure(); plt.plot(df.values, label='filter output')
        #plt.plot([thresh_filter]*len(df), label='thresh_filterold')

    # find start indices
    all_indices = np.where(df.values<thresh_filter)[0]
    all_times = [tango_df.timestamp[i] for i in all_indices]

    start_times = [all_times[0]]
    start_indices = [all_indices[0]]
    for s, i in zip(all_times, all_indices):
        if s > start_times[-1] + 10: # assume that start indices cannot be closer than 10 seconds.
            start_times.append(s)
            start_indices.append(i)
    return start_times, start_indices

def find_end_times(tango_df, plot=False):
    return find_start_times(tango_df, thresh_filter=-0.2, 
                            pattern=[-1, -1, 1, 1, 1, 1], plot=plot)

def find_calibration_indices(tango_df, start_times, start_indices, max_length = 0.01):
    # find calibration indices
    # valid indices are close-to-zero length indices before the start times. 
    calibration_indices = {}

    # weird name inconsistency because in terms of calibration start_time is the end-time. 
    for end_time, end_index in zip(start_times, start_indices):
        num_min = 5
        num_max = min(100, end_index)
        for num_indices in range(num_min, num_max):
            if tango_df.iloc[end_index - num_indices].length > max_length:
                break

        start_index = end_index - num_indices
        start_time = tango_df.iloc[start_index].timestamp
        calibration_indices[start_index] = [start_time, end_time]
    return calibration_indices


# testing! 
test_index = range(100)
test_df = pd.DataFrame(index=test_index, columns=["length", "timestamp"])
start_index_test = [10, 50] 
duration = 20
lengths = np.zeros(100)
for s_idx in start_index_test:
    lengths[s_idx:s_idx+duration] = 1.0
test_df["length"] = lengths
test_df["timestamp"] = test_index
        
start_times, start_indices = find_start_times(test_df)
end_times, end_indices = find_end_times(test_df)
np.testing.assert_allclose(start_indices, start_index_test)
plt.figure()
plt.plot(test_df.timestamp, test_df.length)
for s in start_times:
    plt.plot([s, s], [test_df.length.min(), test_df.length.max()], label='t0', color='orange')
for s in end_times:
    plt.plot([s, s], [test_df.length.min(), test_df.length.max()], label='t0', color='black')

In [ ]:
tango_df = data_df.loc[data_df.system_id=="Tango"]
tango_df = add_length(tango_df)
start_times, start_indices = find_start_times(tango_df, plot=False)
plt.figure()
plt.plot(tango_df.timestamp, tango_df.length)
for s in start_times:
    plt.plot([s, s], [tango_df.length.min(), tango_df.length.max()], label='t0', color='orange')
end_times, end_indices = find_end_times(tango_df, plot=False)
for s in end_times:
    plt.plot([s, s], [tango_df.length.min(), tango_df.length.max()], label='t0', color='black')
    
calibration_indices = find_calibration_indices(tango_df, start_indices=start_indices, start_times=start_times)
    
for idx, [start_time, end_time] in calibration_indices.items():
    plt.plot([start_time, start_time], [tango_df.length.min(), tango_df.length.max()], 
              color='green')

In [ ]:
def calibrate(row, calib_dict, calib_type):
    if row.system_id == 'Tango':
        return 0.0
    offset = calib_dict[row.anchor_name][calib_type]
    return row.distance - offset

plot=False # this can somehow become very slow. 

for j, start_index in enumerate(sorted(calibration_indices.keys())):

    start_time, end_time = calibration_indices[start_index]
    # calculate offsets
    rtt_df = data_df[data_df.system_id=='RTT']
    
    calib_dict = dict()
    
    if plot:
        fig, axs = plt.subplots(1, len(rtt_df.anchor_id.unique()), sharex=True, sharey=False)
        fig.set_size_inches(15, 5)
        fig.suptitle('calibration from {} to {}'.format(start_time, end_time))
    
    for i, (anchor_name, df) in enumerate(rtt_df.groupby('anchor_name')):
        df = df[(df.timestamp >= start_time) & (df.timestamp <= end_time)]
        
        errors = df.distance.values - df.distance_tango.values
        
        if plot:
            axs[i].set_title('{} N={}'.format(anchor_name, len(errors)))
            axs[i].hist(errors)
        
        # mean calibration parameters
        calib_dict[anchor_name] = dict(mean=np.mean(errors),  median=np.median(errors))
        
    # calibrate distances
    data_df.loc[:, 'distance_mean_{}'.format(j)] = data_df.apply(lambda row: calibrate(row, calib_dict, 'mean'), axis=1)
    data_df.loc[:, 'distance_median_{}'.format(j)] = data_df.apply(lambda row: calibrate(row, calib_dict, 'median'), axis=1)

## Calibrate using all measurements

In [ ]:
rtt_df = data_df[data_df.system_id=='RTT']
    
calib_dict = {} 
    
#fig, axs = plt.subplots(1, len(rtt_df.anchor_id.unique()), sharex=True, sharey=True)
#fig.set_size_inches(15, 5)
#fig.suptitle('calibration using all measurements')
for i, (anchor_name, df) in enumerate(rtt_df.groupby('anchor_name')):
    print(i, anchor_name)
    errors = df.distance.values - df.distance_tango.values
    #axs[i].hist(errors)
    #axs[i].set_title(anchor_name)
    calib_dict[anchor_name] = dict(all_mean=np.mean(errors), all_median=np.median(errors))
data_df.loc[:, 'distance_mean_all'.format(j)] = data_df.apply(lambda row: calibrate(row, calib_dict, 'all_mean'), axis=1)
data_df.loc[:, 'distance_median_all'.format(j)] = data_df.apply(lambda row: calibrate(row, calib_dict, 'all_median'), axis=1)

## Calibrate using static positions

In [ ]:
# TODO use calib1-calib5 datasets for calibration. 

## Plot results

In [ ]:
id_vars = [c for c in data_df.columns if c[:8]!='distance']
plot_df = data_df.melt(id_vars=id_vars, 
                       var_name="distance_type", 
                       value_name="distance")
plot_df = plot_df[plot_df.system_id=='RTT']
plot_df.sort_values(['timestamp', 'anchor_name'], inplace=True)
plot_df.reset_index(inplace=True, drop=True)
plot_df.loc[:, 'distance'] = plot_df.distance.astype(np.float32)
plot_df.loc[:, 'timestamp'] = plot_df.timestamp.astype(np.float32)
plot_df.head()

In [ ]:
test_anchor_name = "RTT 0"
sns.lineplot(x='timestamp', y='distance', hue='distance_type', 
             data=plot_df[plot_df.anchor_name==test_anchor_name])

In [ ]:
fg = sns.FacetGrid(plot_df, col='anchor_name', col_wrap=3, hue='distance_type', col_order=sorted(plot_df.anchor_name.unique()), 
                   hue_order=['distance', 'distance_tango', 'distance_median_0', 'distance_median_1', 'distance_median_all'])
fg.map(sns.lineplot, 'timestamp', 'distance')
plt.gcf().set_size_inches(15, 15)
plt.legend()

In [ ]:
from plotting_tools import plot_cdf_raw
from itertools import cycle

colors = sns.color_palette('deep')

fig, axarr = plt.subplots(2, 4)
fig.set_size_inches(15, 10)
axarr = axarr.reshape((-1, ))
for i, (anchor_name, df) in enumerate(plot_df.sort_values("anchor_name").groupby("anchor_name")):
    
    axarr[i].set_title(anchor_name)
    df = df.sort_values("timestamp")
    gt_df = df[df.distance_type=="distance_tango"]
    
    color_cycle = cycle(colors)
    for distance_type in sorted(df.distance_type.unique()):
        if distance_type == "distance_tango" or distance_type == "distance_gt":
            continue
        meas_df = df[df.distance_type==distance_type]
        
        np.testing.assert_allclose(meas_df.timestamp.values, gt_df.timestamp.values)
        
        errors = np.abs(meas_df.distance.values - gt_df.distance.values)
        plot_cdf_raw(errors, ax=axarr[i], color=next(color_cycle), label=distance_type)
        axarr[i].set_ylabel('')
axarr[i].legend(loc='lower left', bbox_to_anchor=[1.0, 0])
for j in range(i+1, len(axarr)):
    axarr[j].axis('off')

In [ ]:
from plotting_tools import plot_cdf_raw
from itertools import cycle

colors = sns.color_palette('deep')

fig, axarr = plt.subplots(2, 4)
fig.set_size_inches(15, 10)
axarr = axarr.reshape((-1, ))
for i, (anchor_name, df) in enumerate(plot_df.sort_values("anchor_name").groupby("anchor_name")):
    axarr[i].set_title(anchor_name)
    df = df.sort_values("timestamp")
    
    color_cycle = cycle(colors)
    for distance_type in sorted(df.distance_type.unique()):
        meas_df = df[df.distance_type==distance_type]
        axarr[i].plot(meas_df.timestamp.values, meas_df.distance.values, color=next(color_cycle), label=distance_type)
axarr[i].legend(loc='lower left', bbox_to_anchor=[1.0, 0])
for j in range(i+1, len(axarr)):
    axarr[j].axis('off')

In [ ]:
from itertools import cycle

colors = sns.color_palette('deep')

fig, axarr = plt.subplots(2, 4, sharey=True, sharex=True)
fig.set_size_inches(15, 10)
axarr = axarr.reshape((-1, ))
for i, (anchor_name, df) in enumerate(plot_df.sort_values("anchor_name").groupby("anchor_name")):
    
    axarr[i].set_title(anchor_name)
    df = df.sort_values("timestamp")
    gt_df = df[df.distance_type=="distance_tango"]
    
    color_cycle = cycle(colors)
    meas_df = df[df.distance_type=="distance_median_all"]
        
    assert np.allclose(meas_df.timestamp.values, gt_df.timestamp.values)
        
    errors = np.abs(meas_df.distance.values - gt_df.distance.values)
    rssis = meas_df.rssi.values
        
    axarr[i].scatter(rssis, errors)
for j in range(i+1, len(axarr)):
    axarr[j].axis('off')

# Position processing

In [ ]:
plt.figure()
data = data_df[data_df.system_id=="Tango"]
sns.scatterplot(data=data, x='timestamp', y='px', linewidth=0.0, label='x')
sns.scatterplot(data=data, x='timestamp', y='py', linewidth=0.0, label='y')
sns.scatterplot(data=data, x='timestamp', y='pz', linewidth=0.0, label='z')
plt.legend()

In [ ]:
plt.figure()

tango_df = data_df.loc[data_df.system_id=="Tango"]

sns.scatterplot(data=tango_df, x='px', y='py', hue='timestamp', linewidth=0.0)
sns.scatterplot(data=anchors_df, x='px', y='py', hue='anchor_name', linewidth=0.0, style='system_id', legend=False)
plt.arrow(0, 0, 1, 0, color='red', head_width=0.2)
plt.arrow(0, 0, 0, 1, color='green', head_width=0.2)
delta = 0.1
for i, a in anchors_df.iterrows():
    plt.annotate(a.anchor_name, (a.px+delta, a.py+delta))
plt.legend(loc='lower right', bbox_to_anchor=[1.0, 0.0])
plt.axis('equal')
plt.title(datafile)

In [ ]:
from trajectory import Trajectory

start_times, start_indices = find_start_times(tango_df, plot=False)
end_times, end_indices = find_end_times(tango_df, plot=False)

# TODO read the correct trajectory
trajectory = Trajectory(dim=2, model='full_bandlimited')
trajectory.set_coeffs(coeffs=np.array([[0, 2, 0], [0, 0, 2]]))

df_predicted = pd.DataFrame(index=range(len(tango_df)), columns=["px", "py", "pz", "timestamp", "model_timestamp"])
df_predicted.loc[:, "timestamp"] = tango_df.timestamp
df_predicted.reset_index(inplace=True, drop=True)

previous_end = 0
for i in range(len(start_times)):
    
    i0 = start_indices[i]
    iN = end_indices[i]
    
    indices = np.arange(i0, iN)
    lengths = tango_df.length.values[indices]
    
    print('sum of lengths', np.sum(lengths))
    print('should be:', 2 * np.pi * 2) # 
    
    distances = np.cumsum(lengths)
    times, _, _  = trajectory.get_times_from_distances(arbitrary_distances=distances, plot=False)
    if len(times) < len(distances):
        print('tried to go until {} but ended at {}'.format(len(distances), len(times)))
        iN = i0 + len(times)
    
    basis = trajectory.get_basis(times=times)
    points = trajectory.get_sampling_points(basis=basis)
    
    plt.figure()
    trajectory.plot(basis=basis)
    plt.scatter(points[0, :], points[1, :])
    

    # TODO rotate and scale to fit the Tango measurements.
    
    print(i0, iN)
    
    df_predicted.loc[i0:iN-1, ["px", "py"]] = points.T
    df_predicted.loc[i0:iN-1, "model_timestamp"] = times
    
    # fill also the stationary points:
    df_predicted.loc[previous_end:i0, ["px", "py"]] = points.T[0]
    df_predicted.loc[iN:-1, ["px", "py"]] = points.T[-1]
    previous_end = iN
df_predicted.loc[:, "pz"] = 0.0

In [ ]:
plt.figure()
sns.scatterplot(data=df_predicted, x='px', y='py', hue='timestamp', 
               linewidth=0.0)
df_predicted.head()

## More processing, in meta-stage...

In [ ]:
from evaluate_dataset import add_median_raw, add_gt_raw

print(datafile)
data_df = read_dataset(datafile)
data_df.head()

# Our way

#data_df = add_median_raw(data_df)
#t1=time.time()
#data_df = add_gt_raw(data_df, t_window=0.1)

# Alternative way. 

### IMPORTANT: this is not centered. Our own implementation add_median_raw is centered. 
### However ours is much much slower...

import datetime
data_df.sort_values("timestamp", inplace=True)
datetimes = [datetime.datetime.fromtimestamp(t/1000.0) for t in data_df.timestamp]
data_df.index = [pd.Timestamp(datetime) for datetime in datetimes]
for anchor_id in data_df.anchor_id.unique():
    anchor_df = data_df[data_df.anchor_id==anchor_id]
    rolling_data = anchor_df['distance'].rolling('1s', min_periods=1, center=False)
    data_df.loc[data_df.anchor_id==anchor_id, "distance_mean"] = rolling_data.mean()
    data_df.loc[data_df.anchor_id==anchor_id, "distance_median"] = rolling_data.median() 
data_df.index = range(len(data_df))

In [ ]:
fg = sns.FacetGrid(data_df, col='anchor_name', col_wrap=4, col_order=sorted(data_df.anchor_name.unique()))
fg.map(plt.plot, 'seconds', 'distance')
fg.map(plt.plot, 'seconds', 'distance_gt')

def plot_distances(data_df):
    import itertools
    colors = itertools.cycle(plt.get_cmap('tab10').colors)
    rtt_ids = anchors_df[anchors_df.system_id==rtt_system_id].anchor_id.unique()
    fig, axs = plt.subplots(1, len(rtt_ids), sharey=True)
    fig.set_size_inches(15, 5)
    for ax, anchor_id in zip(axs, rtt_ids):
        color = next(colors)
        data = data_df[data_df.anchor_id==anchor_id]
        anchor_name = anchors_df.loc[anchors_df.anchor_id==anchor_id, 'anchor_name']
        ax.plot(data.seconds, data.distance_gt, linestyle=':', label=anchor_name, color=color)
        ax.plot(data.seconds, data.distance, linestyle='-', color=color)
        ax.set_ylim(0, 15)
    #sns.scatterplot(data=data_df[data_df.system_id==rtt_system_id], x='seconds', y='distance_error',hue='anchor_id')
plot_distances(data_df)

In [ ]:
parameters = {
    'experiments/robot_test/static1.csv':  np.array([0, 0, 0.2]), # on robot, facing window
    'experiments/robot_test/static2.csv':  np.array([0, 0, 0.1]), # in wooden thing, facing window
    'experiments/robot_test/static3.csv':  np.array([0, 0, 0.1]), # in wooden thing, facing wall
    'experiments/robot_test/static4.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static5.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static6.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static7.csv':  np.array([0, 0, 0.2]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static8.csv':  np.array([0, 0, 0.2]), # in wooden thing on stand, facing wall
}

for i in range(1, 9):
    datafile = 'experiments/robot_test/static{}.csv'.format(i)
    position = parameters[datafile]

    data_df = read_dataset(datafile)
    plot_distances(data_df)
    name = datafile.split('/')[-1]
    plt.suptitle(name)

In [ ]:
import os
folder = 'experiments/robot_test/'
file_list = os.listdir(folder)
plt.figure()

end_points_lines_room = np.array([
    [5.046, 5.615, 0.0],
    [4.869, 4.231, 0.0],
    [5.870, 1.830, 0.0],
    [4.254, 1.567, 0.0],
    [2.924, 1.867, 0.0],
    [1.198, 1.416, 0.0]]).T
start_point_room = np.array([1.034, 5.410, 0.0]).reshape((3, 1))

end_points_lines = convert_room_to_robot(end_points_lines_room)
start_point = convert_room_to_robot(start_point_room)

for datafile in sorted(file_list):
    if datafile[-4:] != '.csv':
        continue
    if (datafile[:8] != 'straight') and (datafile[:5] != 'calib') or (datafile[:10] == 'calibratio'):
        continue
    data_df = pd.read_csv(folder + datafile)
    #data_df.loc[:, 'seconds'] = (data_df.timestamp.values - data_df.timestamp.min()) / 1000. # in seconds
    data = data_df[data_df.system_id==tango_system_id]
    sns.scatterplot(data=data, x='px', y='py', label=datafile, linewidth=0.0)

plt.scatter(*start_point[:2], color='black', s=10.0)
for end_point in end_points_lines.T:
    print(end_point)
    plt.scatter(*end_point[:2], color='black', s=5.0)
    plt.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color='black')
    
plt.axis('equal')
plt.legend(loc='lower left', bbox_to_anchor=[0.8, 0])
#plt.legend()
#fill_ground_truth(data_df, current_gt=position)

In [ ]:
for datafile in sorted(file_list):
    if datafile[-4:] != '.csv':
        continue
    if datafile[:6] == 'static' or datafile[:5] == 'calib':
        continue
    else:
        data_df = pd.read_csv(folder + datafile)
        data = data_df[data_df.system_id==tango_system_id]
        plt.figure()
        sns.scatterplot(data=data, x='px', y='py', label=datafile, linewidth=0.0)